In [20]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetV2B2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization, Activation, Dropout
import numpy as np
from PIL import Image
import os

# Class names
class_names = [
    "Asian-Green-Bee-Eater",
    "Brown-Headed-Barbet",
    "Cattle-Egret",
    "Common-Kingfisher",
    "Common-Myna",
    "Common-Rosefinch",
    "Common-Tailorbird",
    "Coppersmith-Barbet",
    "Forest-Wagtail",
    "Gray-Wagtail",
    "Hoopoe",
    "House-Crow",
    "Indian-Grey-Hornbill",
    "Indian-Peacock",
    "Indian-Pitta",
    "Indian-Roller",
    "Jungle-Babbler",
    "Northern-Lapwing",
    "Red-Wattled-Lapwing",
    "Ruddy-Shelduck",
    "Rufous-Treepie",
    "Sarus-Crane",
    "White-Breasted-Kingfisher",
    "White-Breasted-Waterhen",
    "White-Wagtail"
]

# Build the model
base_model = EfficientNetV2B2(
    weights='imagenet',
    include_top=False,
    input_shape=(300, 300, 3)
)

base_model.trainable = False

model = Sequential([
    base_model,
    
    GlobalAveragePooling2D(), 

    Dense(256, name='dense_256'),  
    BatchNormalization(name='bn_256'),  
    Activation('relu'),  
    Dropout(0.3),  

    Dense(128, name='dense_128'), 
    BatchNormalization(name='bn_128'),
    Activation('relu'),  
    Dropout(0.3),

    Dense(25, activation='softmax', name='dense_out')  
])

# Load weights manually
import h5py
with h5py.File('weights.h5', 'r') as f:
    # Dense 256
    dense_weights = f['layers']['dense']['vars']['0'][:]
    dense_bias = f['layers']['dense']['vars']['1'][:]
    model.get_layer('dense_256').set_weights([dense_weights, dense_bias])
    
    # Batch norm 256
    bn_gamma = f['layers']['batch_normalization']['vars']['0'][:]
    bn_beta = f['layers']['batch_normalization']['vars']['1'][:]
    bn_mean = f['layers']['batch_normalization']['vars']['2'][:]
    bn_var = f['layers']['batch_normalization']['vars']['3'][:]
    model.get_layer('bn_256').set_weights([bn_gamma, bn_beta, bn_mean, bn_var])
    
    # Dense 128
    dense1_weights = f['layers']['dense_1']['vars']['0'][:]
    dense1_bias = f['layers']['dense_1']['vars']['1'][:]
    model.get_layer('dense_128').set_weights([dense1_weights, dense1_bias])
    
    # Batch norm 128
    bn1_gamma = f['layers']['batch_normalization_1']['vars']['0'][:]
    bn1_beta = f['layers']['batch_normalization_1']['vars']['1'][:]
    bn1_mean = f['layers']['batch_normalization_1']['vars']['2'][:]
    bn1_var = f['layers']['batch_normalization_1']['vars']['3'][:]
    model.get_layer('bn_128').set_weights([bn1_gamma, bn1_beta, bn1_mean, bn1_var])
    
    # Dense out
    dense2_weights = f['layers']['dense_2']['vars']['0'][:]
    dense2_bias = f['layers']['dense_2']['vars']['1'][:]
    model.get_layer('dense_out').set_weights([dense2_weights, dense2_bias])

print("Model built and weights loaded!")

# Predict on the image
if os.path.exists('Painted_Bunting_male_Birdhsare_Tim_Hopwood.jpg'):
    img = Image.open('Painted_Bunting_male_Birdhsare_Tim_Hopwood.jpg').convert('RGB')
    img = img.resize((300, 300))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # add batch
    
    pred = model.predict(img_array)
    predicted_class = np.argmax(pred)
    confidence = np.max(pred)
    predicted_class_name = class_names[predicted_class]
    print(f"Predicted class: {predicted_class} ({predicted_class_name}), Confidence: {confidence:.4f}")
else:
    print("Image file not found.")

Model built and weights loaded!
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Predicted class: 17 (Northern-Lapwing), Confidence: 0.7376
